In [1]:
import time
import cv2
import numpy as np
from skimage.util import img_as_float
from skimage.util import img_as_ubyte


def show_in_moved_window(win_name, img, x, y):
    """
    Show an image in a window, where the position of the window can be given
    """
    cv2.namedWindow(win_name)
    cv2.moveWindow(win_name, x, y)
    cv2.imshow(win_name,img)


def capture_from_camera_and_show_images():
    print("Starting image capture")

    print("Opening connection to camera")
    url = 0
    use_droid_cam = False
    if use_droid_cam:
        url = "http://192.168.1.120:4747/video"
    cap = cv2.VideoCapture(url)
    # cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    print("Starting camera loop")
    # Get first image
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame")
        exit()

    # Transform image to gray scale and then to float, so we can do some processing
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = img_as_float(frame_gray)

    # To keep track of frames per second
    start_time = time.time()
    n_frames = 0
    stop = False
    while not stop:
        ret, new_frame = cap.read()
        if not ret:
            print("Can't receive frame. Exiting ...")
            break

        # Transform image to gray scale and then to float, so we can do some processing
        new_frame_gray = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
        new_frame_gray = img_as_float(new_frame_gray)

        # Compute difference image
        dif_img = np.abs(new_frame_gray - frame_gray)

        # Keep track of frames-per-second (FPS)
        n_frames = n_frames + 1
        elapsed_time = time.time() - start_time
        fps = int(n_frames / elapsed_time)

        # Put the FPS on the new_frame
        str_out = f"fps: {fps}"
        font = cv2.FONT_HERSHEY_COMPLEX
        cv2.putText(new_frame, str_out, (100, 100), font, 1, 255, 1)

        # Display the resulting frame
        show_in_moved_window('Input', new_frame, 0, 10)
        show_in_moved_window('Input gray', new_frame_gray, 600, 10)
        show_in_moved_window('Difference image', dif_img, 1200, 10)

        # Old frame is updated
        frame_gray = new_frame_gray

        if cv2.waitKey(1) == ord('q'):
            stop = True

    print("Stopping image loop")
    cap.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    capture_from_camera_and_show_images()

Starting image capture
Opening connection to camera
Starting camera loop
Stopping image loop


In [3]:
import time
import cv2
import numpy as np
from skimage.util import img_as_float, img_as_ubyte

def show_in_moved_window(win_name, img, x, y):
    """
    Show an image in a window, where the position of the window can be given
    """
    cv2.namedWindow(win_name)
    cv2.moveWindow(win_name, x, y)
    cv2.imshow(win_name, img)

def capture_from_camera_and_detect_changes(alpha=0.95, T=0.1, A=0.05):
    print("Starting image capture")

    # Connect to camera
    print("Opening connection to camera")
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    # Acquire initial background image
    print("Capturing background image")
    ret, background_frame = cap.read()
    if not ret:
        print("Can't receive frame")
        exit()
    
    # Convert background image to grayscale and floating-point
    background_gray = cv2.cvtColor(background_frame, cv2.COLOR_BGR2GRAY)
    background_gray = img_as_float(background_gray)
    
    print("Starting camera loop")
    start_time = time.time()
    n_frames = 0
    stop = False
    
    while not stop:
        ret, new_frame = cap.read()
        if not ret:
            print("Can't receive frame. Exiting ...")
            break
        
        # Convert new frame to grayscale and floating-point
        new_frame_gray = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
        new_frame_gray = img_as_float(new_frame_gray)
        
        # Compute difference image
        dif_img = np.abs(new_frame_gray - background_gray)
        
        # Create binary image using threshold T
        binary_img = dif_img > T
        
        # Compute the total number of foreground pixels
        F = np.sum(binary_img)
        total_pixels = binary_img.size
        percentage_F = F / total_pixels
        
        # Check if the number of changed pixels exceeds the alert value
        if percentage_F > A:
            str_out = "Change Detected!"
            color = (0, 0, 255)  # Red color for alert
        else:
            str_out = "No Change"
            color = (0, 255, 0)  # Green color for no change
        
        # Put text on the new frame
        font = cv2.FONT_HERSHEY_COMPLEX
        cv2.putText(new_frame, str_out, (100, 100), font, 1, color, 1)
        
        # Display images
        show_in_moved_window('Input', new_frame, 0, 10)
        show_in_moved_window('Background', img_as_ubyte(background_gray), 600, 10)
        show_in_moved_window('Difference Image', img_as_ubyte(dif_img), 1200, 10)
        show_in_moved_window('Binary Image', img_as_ubyte(binary_img), 1800, 10)
        
        # Update background image
        background_gray = alpha * background_gray + (1 - alpha) * new_frame_gray
        
        # Calculate FPS
        n_frames += 1
        elapsed_time = time.time() - start_time
        fps = int(n_frames / elapsed_time)
        cv2.putText(new_frame, f"fps: {fps}", (100, 50), font, 1, (255, 255, 255), 1)
        
        # Check for exit key
        if cv2.waitKey(1) == ord('q'):
            stop = True

    print("Stopping image loop")
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    capture_from_camera_and_detect_changes()


Starting image capture
Opening connection to camera
Capturing background image
Starting camera loop
Stopping image loop
